# Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Loading Related Data Sets

In [3]:
Psy = pd.read_csv('/content/drive/MyDrive/sai/b13 youtube/Youtube01-Psy.csv')
Katy = pd.read_csv('/content/drive/MyDrive/sai/b13 youtube/Youtube02-KatyPerry.csv')
Eminem = pd.read_csv('/content/drive/MyDrive/sai/b13 youtube/Youtube04-Eminem.csv')
Shakira = pd.read_csv('/content/drive/MyDrive/sai/b13 youtube/Youtube05-Shakira.csv')
LMFAO = pd.read_csv('/content/drive/MyDrive/sai/b13 youtube/Youtube03-LMFAO.csv')

# Data Cleaning

# Concatinating five datasets

In [4]:
df = pd.concat([Shakira, Eminem, Katy, Psy, LMFAO])
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,2015-05-29T02:30:18.971000,Nice song﻿,0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,2015-05-29T00:14:48.748000,I love song ﻿,0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,2015-05-28T21:00:08.607000,I love song ﻿,0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,2015-05-28T20:47:12.193000,"860,000,000 lets make it first female to reach...",0
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,2015-05-28T17:08:29.827000,shakira is best for worldcup﻿,0


# Dropping Date Column

In [5]:
df.drop('DATE', axis=1, inplace=True)
df.head()

,COMMENT_ID,AUTHOR,CONTENT,CLASS
0,z13lgffb5w3ddx1ul22qy1wxspy5cpkz504,dharma pal,Nice song﻿,0
1,z123dbgb0mqjfxbtz22ucjc5jvzcv3ykj,Tiza Arellano,I love song ﻿,0
2,z12quxxp2vutflkxv04cihggzt2azl34pms0k,Prìñçeśś Âliś Łøvê Dømíñø Mâđiś™ ﻿,I love song ﻿,0
3,z12icv3ysqvlwth2c23eddlykyqut5z1h,Eric Gonzalez,"860,000,000 lets make it first female to reach...",0
4,z133stly3kete3tly22petvwdpmghrlli,Analena López,shakira is best for worldcup﻿,0


# Finding the Shape of the Data Set

In [6]:
df.shape

(1956, 4)

In [7]:
data = df[['CONTENT','CLASS']]

In [8]:
data.shape

(1956, 2)

In [9]:
import re

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences\
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

In [12]:
data = data[data.CLASS != "Neutral"]
data['CONTENT'] = data['CONTENT'].apply(lambda x: x.lower())
data['CONTENT'] = data['CONTENT'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['CLASS'] == 0].size)
print(data[ data['CLASS'] == 1].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['CONTENT'].values)
X = tokenizer.texts_to_sequences(data['CONTENT'].values)
X = pad_sequences(X)

1902
2010


In [13]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 195, 128)          256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 195, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
Y = pd.get_dummies(data['CLASS']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(1310, 195) (1310, 2)
(646, 195) (646, 2)


In [15]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 2)

Epoch 1/10
41/41 - 41s - loss: 0.5920 - accuracy: 0.7130 - 41s/epoch - 993ms/step
Epoch 2/10
41/41 - 33s - loss: 0.3528 - accuracy: 0.8641 - 33s/epoch - 798ms/step
Epoch 3/10
41/41 - 31s - loss: 0.2409 - accuracy: 0.9038 - 31s/epoch - 760ms/step
Epoch 4/10
41/41 - 34s - loss: 0.1951 - accuracy: 0.9183 - 34s/epoch - 834ms/step
Epoch 5/10
41/41 - 30s - loss: 0.1323 - accuracy: 0.9412 - 30s/epoch - 738ms/step
Epoch 6/10
41/41 - 32s - loss: 0.1098 - accuracy: 0.9550 - 32s/epoch - 785ms/step
Epoch 7/10
41/41 - 30s - loss: 0.0978 - accuracy: 0.9626 - 30s/epoch - 736ms/step
Epoch 8/10
41/41 - 34s - loss: 0.0789 - accuracy: 0.9649 - 34s/epoch - 840ms/step
Epoch 9/10
41/41 - 32s - loss: 0.0804 - accuracy: 0.9679 - 32s/epoch - 776ms/step
Epoch 10/10
41/41 - 30s - loss: 0.0629 - accuracy: 0.9748 - 30s/epoch - 739ms/step


## Predictions


In [17]:
model.predict(X_test).shape

21/21 [==============================] - 1s 54ms/step


(646, 2)

In [18]:
X_test.shape

(646, 195)

In [22]:
twt = ['Meetings: Because none of us is as dumb as all of us.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input


twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt)
if(np.argmax(sentiment) == 0):
    print("Not Spam")
elif (np.argmax(sentiment) == 1):
    print("Spam")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0   98 1949   14   87   13   60  811   60   47   14   87]]
1/1 [==============================] - 0s 21ms/step
Not Spam


In [23]:

twt = ['bahjjdfkjdgnjgf|']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input


twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt)
if(np.argmax(sentiment) == 0):
    print("Not Spam")
elif (np.argmax(sentiment) == 1):
    print("Spam")

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
1/1 [==============================] - 0s 22ms/step
Spam
